In [ ]:
import json
import os
import random
import sys

In [ ]:
from typing import Iterator, Mapping

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

from typing import Any, NamedTuple

In [ ]:
from PIL import Image
from numpy import asarray
import jax.numpy as jnp
import jax
from jax import jit
from jax import random

In [67]:
import transferlearningresnet as resnetModel

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [68]:
def wordDictionary(path):
  tokens_series = pd.read_csv(path, delimiter='\s+', header=0, names=['Token', 'Frequency'], usecols=[0]).squeeze()
  # tokens_series.index = tokens_series.index+1
  tokens = {tokens_series.values[i]: tokens_series.index[i] for i in range(tokens_series.shape[0])}
  return tokens

In [69]:
# fashion_tokens = wordDictionary('gdrive/MyDrive/Colab Notebooks/Capstone Project/Outfit_Processing/Data/final_word_dict.txt')
# fashion_tokens
# len(fashion_tokens)
# 2755
# list(fashion_tokens.values())[-1]

In [85]:
def to_sequence_Array(set_info, InceptionV3_InputImageDim, tokenDic, dataPath):

  """Builds a SequenceExample proto for an outfit.
  """
  set_id = set_info['set_id']
  image_data = []
  image_ids = []
  caption_data = []
  caption_ids = []
  
  dict_len = len(tokenDic)

  outfit_path = os.path.join(dataPath, 'Outfits', str(set_id))

  if not(os.path.exists(outfit_path)):
    return
    
  def fixedLength_ImageAndCaption_Seq(img_caption_Seq, fixedSeqLength):
    imgSeq, captionSeq = img_caption_Seq
    assert len(imgSeq) == len(captionSeq)
    
    l = len(imgSeq)
    if l > fixedSeqLength:
      return imgSeq[:fixedSeqLength], captionSeq[:fixedSeqLength]
    elif l < fixedSeqLength:
      return imgSeq+[imgSeq[-1]]*(fixedSeqLength-l), captionSeq+[captionSeq[-1]]*(fixedSeqLength-l)
    else:
      return imgSeq, captionSeq

  def imageProcessing(filename):
    image = Image.open(filename)
    img_resized = image.resize(InceptionV3_InputImageDim)
    img_numpy =  asarray(img_resized)
    return img_numpy

  def captionProcessing(caption):

    def oneHotEncoding(tokenIds, vectorLength):
      zeroVector = [0]*vectorLength
      oneHotVectors = [zeroVector[:i]+[1]+zeroVector[i+1:] for i in tokenIds]
      oneHotVectorsArray = np.array(oneHotVectors, dtype=jnp.float32)
      return oneHotVectorsArray

    token_ids = [tokenDic.get(word, dict_len) for word in caption.split()]
    if not(len(token_ids)):
      token_ids = [dict_len]
    oneHotVectors = oneHotEncoding(token_ids, dict_len+1)
    oneHotEmbedding = np.sum(oneHotVectors, axis=0)
    return oneHotEmbedding
  
  for image_info in set_info['items']:
    
    imagefile = os.path.join(outfit_path, str(image_info['index']) + '.jpg')
    image = imageProcessing(imagefile)
    image_data.append(image)

    caption = image_info['name']
    oneHotEncoding = captionProcessing(caption)
    caption_data.append(oneHotEncoding)

  image_data, caption_data = fixedLength_ImageAndCaption_Seq((image_data, caption_data), 8)

  caption_feature = np.array(caption_data)

  # Feature extraction using transfer learning (resnet)
  image_np_array = np.array(image_data)
  model = resnetModel.getResNetModel(image_np_array.shape[1:])
  image_feature = model.predict(image_np_array)
  
  return (image_feature, caption_feature)

In [71]:
# json_file = 'gdrive/MyDrive/Colab Notebooks/Capstone Project/Outfit_Processing/Data/outfit.json'

In [72]:
# all_sets = json.load(open(json_file))
# all_sets[0]

In [86]:
def to_sequence_Tensor(outfits, InceptionV3_InputImageDim, tokenDic, path):

  outfitSequencesImage = []
  outfitSequencesCaption = []

  for outfit in outfits:
    img_caption_seq = to_sequence_Array(outfit, InceptionV3_InputImageDim, tokenDic, path)
    if img_caption_seq:
      image_seq, caption_seq = img_caption_seq
      outfitSequencesImage.append(image_seq)
      outfitSequencesCaption.append(caption_seq)

  return {'outfitSequencesImage' : jnp.array(outfitSequencesImage, dtype=jnp.float32), 'outfitSequencesCaption' : jnp.array(outfitSequencesCaption, dtype=jnp.float32)}

In [87]:
# path = 'gdrive/MyDrive/Colab Notebooks/Capstone Project/Outfit_Processing/Data/'
# sequence_Tensor = to_sequence_Tensor(all_sets, (224,224), fashion_tokens, path)

In [88]:
# sequence_Tensor['outfitSequencesImage'].shape, sequence_Tensor['outfitSequencesCaption'].shape

((3, 8, 100352), (3, 8, 2756))

CACHE

In [ ]:
# ((1700, 8, 100352), (1700, 8, 2756))

# ((5, 8, 100352), (5, 8, 2756)) 1
# ((5, 8, 100352), (5, 8, 2756)) 2
# ((5, 8, 100352), (5, 8, 2756)) 3

# (8, 100352)  --> wRNN*Img --> (8, 512)   RNN_E
# (8, 100352)  --> wVS*Img -->  (8, 512)   VS_E
# (8, 2756)   --> Caption

# RNN_E --> Bi-LSTM model --> obj (B+F)
# VS_E, Caption   --> VS_Model (VS_E, Caption_512)  --> obj_VS

# Obj = Module (obj (B+F) + obj_VS)

# gObj =  Gradient(Obj)


# SGD, Adam --> gObj  --> tune our weights

# Obj_Trained

# Prediction



In [ ]:
# [[0 0 0 1],
#  [0 1 0 0]
#  [1 0 0 0]]


#  caption_w@[1 1 0 1]

In [ ]:
# imgSet = sequence_Tensor['outfitSequencesImage'][0]
# imgSet.shape

In [ ]:
# feature_vector = resnetModel.model.predict(np.array(imgSet))
# feature_vector.shape

In [82]:
# image_caption_array = to_sequence_Array(all_sets[0], (224,224), fashion_tokens, path)

gdrive/MyDrive/Colab Notebooks/Capstone Project/Outfit_Processing/Data/Outfits/214181831


In [83]:
# image_caption_array[0].shape, image_caption_array[1].shape

((8, 100352), (8, 2756))

In [ ]:
# Image.fromarray(np.array(image_caption_array['img_seq'][4]))

In [ ]:
# def oneHotEncoding(tokenIds, vectorLength):
#   zeroVector = [0]*vectorLength
#   oneHotVectors = [zeroVector[:i]+[1]+zeroVector[i+1:] for i in tokenIds]
#   return oneHotVectors
  
# # a1 = oneHotEncoding([2,5,8], 10)
# # a1

SUDO INCEPTION_V3 / RESNET  VNCON3_BN5C --> followed by average pooling --> output 512 dimentional 


rank k prediction --> validation matrix (performance)

seimis model (1,2) (2,3) 

In [ ]:
# def inception_v3_sudo(images):
  
#   r = images.shape[1]*images.shape[2]
#   c = 2048
  
#   key = random.PRNGKey(7)
#   W  = random.normal(key, shape=(r,c))

#   X1 = jnp.sum(images, axis=3)
#   X2 = jnp.reshape(X1, (-1, r))
#   inception_output = jnp.matmul(X2, W)

#   return inception_output


In [ ]:
# inception_Output = inception_v3_sudo(image_caption_array['img_seq'])
# inception_Output.shape

In [ ]:
# visual_semantic = {'img_seq': inception_Output, 'caption_seq': a['caption_seq']}

In [ ]:
# jnp.dtype(visual_semantic['img_seq']), jnp.dtype(visual_semantic['caption_seq']), visual_semantic['img_seq'].shape, visual_semantic['caption_seq'].shape

In [ ]:
# inception_Output_batched = jnp.reshape(inception_Output, (inception_Output.shape[0], 1, inception_Output.shape[1]))
# inception_Output_batched.shape

In [ ]:
# all_sets = json.load(open(labels_file))

In [ ]:
# !unzip Data.zip

In [ ]:
# print(filename)
# with open(filename, "rb") as f:
#   encoded_image = f.read()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# image = Image.open('/content/gdrive/MyDrive/Colab Notebooks/Capstone Project/Outfit_Processing/Data/Outfits/120161271/1.jpg')
# # summarize some details about the image
# print(image.format)
# print(image.mode)
# print(image.size)
# # show the image
# image.show()

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot

In [ ]:
# data = image.imread('/content/gdrive/MyDrive/Colab Notebooks/Capstone Project/Outfit_Processing/Data/Outfits/120161271/1.jpg')

# pyplot.imshow(data)

In [ ]:
# a = [2,4,1,2,5]
# b = enumerate(a)
# list(b)

# [2]*3

In [ ]:
# img_jnp = jnp.array([np.array([1,2,3]), np.array([4,5,6])])
# img_jnp

In [ ]:
# a =  [1,2,3]
# a.extend([-1]*0)
# a

    # img_jnp = jnp.array(img_numpy)

In [ ]:
# len(a), a['caption_seq'][0].shape, a['img_seq'][0].shape
# b = [len(a['caption_seq'][i]) for i in range(len(a['caption_seq']))]
# b, len(a['caption_seq'][2][4])
# d = [sum(a['caption_seq'][1][i]) for i in range(len(a['caption_seq'][1]))]
# d
# a['caption_seq']

In [ ]:
# def fixedLengthImageAndCaptionSeq(imgSeq, captionSeq, fixedSeqLength, maxLength):
#   assert len(imgSeq) == len(captionSeq)
#   zeroVector = [0]*(dict_len+1)
#   zeroMatrix = [zeroVector]*maxLength

#   l = len(imgSeq)
#   if l > fixedSeqLength:
#     return imgSeq[:fixedSeqLength], captionSeq[:fixedSeqLength]
#   elif l < fixedSeqLength:
#     return imgSeq+[imgSeq[-1]]*(fixedSeqLength-l), captionSeq+[zeroMatrix]*(fixedSeqLength-l)
#   else:
#     return imgSeq, captionSeq

In [ ]:
# def nullFun():
#   return

# # type(nullFun())
# # nullFun() == None

# if nullFun():
#   print("Not Working")
# else:
#   print('working')